# Instrument classifier

## Imports

In [1]:
import scikitplot as skplt
import soundfile as sf
import numpy as np
#import librosa
#import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import math
import antropy as ant
import seaborn as sns

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (r2_score,
                             roc_auc_score,
                             classification_report,
                             confusion_matrix,
                             roc_curve,
                             accuracy_score,
                             mean_squared_error)
from sklearn.tree import (DecisionTreeClassifier,
                          plot_tree)
from sklearn.ensemble import RandomForestClassifier

from scipy import signal
from scipy.fft import fft, fftfreq, rfftfreq, rfft
from scipy.stats import norm, kurtosis

from pedalboard import LadderFilter, Pedalboard
from soundfile import SoundFile

## Functions

In [2]:
# Centroid
def getCentroid(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    return centroid

# Spread
def getSpread(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    spread = math.sqrt(np.abs(np.sum((frequency - centroid) ** 2 * amplitude) / np.sum(amplitude)))
    return spread

# Skewness
def getSkewness(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    spread = math.sqrt(np.abs(np.sum((frequency - centroid) ** 2 * amplitude) / np.sum(amplitude)))
    skewness = np.sum((frequency - centroid) ** 3 * amplitude) / (spread ** 3 * np.sum(amplitude))
    return skewness

# Kurtosis
def getKurtosis(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    spread = math.sqrt(np.abs(np.sum((frequency - centroid) ** 2 * amplitude) / np.sum(amplitude)))
    kurtosis = np.sum((frequency - centroid) ** 4 * amplitude) / (spread ** 4 * np.sum(amplitude)) - 3
    return kurtosis

# Entropy
def getEntropy(amplitude):
    normalizedAmplitude = amplitude / sum(amplitude)
    entropy = -np.sum(normalizedAmplitude * np.log(normalizedAmplitude)) / np.log(len(amplitude))
    return entropy

# Flatness
def getFlatness(amplitude):
    flatness = np.exp(np.mean(np.log(amplitude))) / np.mean(amplitude)
    return flatness

# Decrease
def getDecrease(amplitude):
    first = amplitude[0]
    amplitude = np.delete(amplitude, 0)
    decrease = np.sum((amplitude - first) / (len(amplitude) - 1)) / np.sum(amplitude)
    return decrease

# Slope
def getSlope(frequency, amplitude):
    slope = sum((frequency - np.mean(frequency)) * (amplitude - np.mean(amplitude))) / np.sum((frequency - np.mean(frequency)) ** 2)
    return slope

# Peak
def getPeak(frequency, amplitude):
    index = np.argmax(np.array(amplitude))
    peak = frequency[index]
    return peak

# Crest
def getCrest(frequency, amplitude):
    index = np.argmax(np.array(amplitude))
    peak = frequency[index]
    crest = peak / np.mean(amplitude)
    return crest

# Rolloff
def getRolloff(amplitude, treshold = 0.95):
    cumsum = 0
    cont = 0
    index = 0
    for frequency in amplitude:
        if cumsum > (amplitude[-1] * treshold):
            index = amplitude[cont]
            break
        cumsum += frequency
        cont += 1
    return index

## Instruments

In [3]:
instruments = ["Kick", "Snare", "Hihat", "Guitar", "Bass", "Vox"]
numSounds = 50

## Audio load

In [4]:
generalPath = "../InstrumentsClassifier/Instruments/"

instrumentsAudio = []
for instrument in instruments:
    instrumentPath = generalPath + instrument + "/"
    sound = []
    
    for i in range(numSounds):
        path = instrumentPath + instrument + "_" + str(i+1) + ".wav"
        audio = SoundFile(path)

        if audio.channels == 1:
            samples, sr = sf.read(path)
            sound.append([samples, sr])
        else:
            samples, sr = sf.read(path)
            sound.append([samples[:,0], sr])
            
    instrumentsAudio.append([instrument, sound])

## Pre-processing

In [6]:
cont = 0
for instrument in instruments:
    sound = []
    
    for i in range(numSounds):        
        amplitude = instrumentsAudio[cont][1][i][0]
        sr = instrumentsAudio[cont][1][i][1]
        time = np.arange(0, len(amplitude), 1) / sr
    
        spectrum = np.abs(rfft(amplitude))
        frequency = rfftfreq(len(amplitude), 1 / sr)
        
        centroid = getCentroid(time, amplitude)
        spread = getSpread(time, amplitude)
        skewness = getSkewness(time, amplitude)
        kurtosis = getKurtosis(time, amplitude)
        decrease = getDecrease(amplitude)
        slope = getSlope(time, amplitude)
        peak = getPeak(time, amplitude)
        crest = getCrest(time, amplitude)
        rolloff = getRolloff(amplitude)
        
        spectralCentroid = getCentroid(frequency, spectrum)
        spectralSpread = getSpread(frequency, spectrum)
        spectralSkewness = getSkewness(frequency, spectrum)
        spectralKurtosis = getKurtosis(frequency, spectrum)
        spectralEntropy = getEntropy(spectrum)
        spectralFlatness = getFlatness(spectrum)
        spectralDecrease = getDecrease(spectrum)
        spectralSlope = getSlope(frequency, spectrum)
        spectralPeak = getPeak(frequency, spectrum)
        spectralCrest = getCrest(frequency, spectrum)
        spectralRolloff = getRolloff(spectrum)
        
        if i == 0 and cont == 2:
            print('Centroid: ', centroid)
            print('Spread: ', spread)
            print('Skewness: ', skewness)
            print('Kurtosis: ', kurtosis)
            print('Decrease: ', decrease)
            print('Slope: ', slope)
            print('Peak: ', peak)
            print('Crest: ', crest)
            print('Rolloff: ', rolloff)
            
            print('Spectral centroid: ', spectralCentroid)
            print('Spectral spread: ', spectralSpread)
            print('Spectral skewness: ', spectralSkewness)
            print('Spectral kurtosis: ', spectralKurtosis)
            print('Spectral entropy: ', spectralEntropy)
            print('Spectral flatness: ', spectralFlatness)
            print('Spectral decrease: ', spectralDecrease)
            print('Spectral slope: ', spectralSlope)
            print('Spectral peak: ', spectralPeak)
            print('Spectral crest: ', spectralCrest)
            print('Spectral rolloff: ', spectralRolloff)
            
    cont += 1

Centroid:  0.0204504982381074
Spread:  0.04673822740731093
Skewness:  1.7819871030534329
Kurtosis:  1.0697189318335933
Decrease:  0.0001906087354839756
Slope:  0.0016578254182013
Peak:  0.01528344671201814
Crest:  -302.5596139990756
Rolloff:  8.344650268554688e-07
Spectral centroid:  7083.6842682779115
Spectral spread:  4154.618244918283
Spectral skewness:  1.1813921330093506
Spectral kurtosis:  1.7588331047888328
Spectral entropy:  0.9324022148366133
Spectral flatness:  0.5036817555028701
Spectral decrease:  0.00035401678018584444
Spectral slope:  -0.0004659476136103953
Spectral peak:  7960.318650421744
Spectral crest:  1660.3824739860304
Spectral rolloff:  0.22494527123562855
